In [1]:
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import requests 

from IPython.core.display import clear_output
from time import time
start_time = time()
# from warnings import warnwarn("Warning Simulation")

from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#import pandas as pd

def get_content_using_Selenium(url):
    DRIVER_PATH = 'C:\\Users\\Nguyen Gia Bao\\Desktop\\chromedriver_win32\\chromedriver'

#     options = Options()
#     options.headless = True
#     options.add_argument("--window-size=1920,1200")

# driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    driver.get(url)
    htmlContent = BeautifulSoup(driver.page_source)
    driver.quit()
    return htmlContent

def request_page(url):
    headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    r = requests.get(url, headers = headers) 
    soup = BeautifulSoup(r.content,'html5lib')
    return soup

# Get list of post
def get_list_of_posts(table, attrs1, attrs2):
    listRental = []
    for row in table.findAll('div', attrs = {'class': attrs1}): 
        quote = {}
        row1 = row.find('div', attrs = {'class': attrs2})
        listRental.append(row1.a['href'])
    return listRental

# Get detail info for each post: Đặc điểm BĐS + Liên hệ
def get_detail_info(table_content,attrs1, attrs2):
    if not table_content:
        print('table_content is empty')
    else:
        key_table = table_content.findAll('div',attrs = {'class':attrs1})
        value_table = table_content.findAll('div',attrs = {'class':attrs2})
        x = {k.text.strip():v.text.strip() for k, v in zip(key_table, value_table)}
        #d = {k: v for k, v in zip(name_list, num_list)
        return x

# Get link den tung post tren tung page
def create_individual_link(url_base, listPosts ):
    listUrl=[]
    for i in listPosts:
        listUrl.append(str(url_base + i))    
    return listUrl

# Get more info: Ma tin dang, loai tin, ngay dang, ngay het han
def get_more_info(tableMoreInfo):
    divList = tableMoreInfo.find_all('div')
    d = {}
    for i in range(len(divList)):
        if i == 1:
            pass
        elif i == 2:
            k = divList[i].span.string
            v = divList[i].find('span', id = 'ltrVipType').text
        else: 
            k,v = divList[i].text.replace('\n','').strip().split(':')
        d[k] = v  
    return d

def print_list(List):
    for i in List:
        print(i)
        
def error_empty(table):
    if not table:
        print('table is empty')

def create_dataFrame(listData):
    data = []
    for i in listData:
        data.append(i[0])
    df = pd.DataFrame(data, columns=['Mã tin đăng',
                                     'Loại hình tin đăng:',
                                     'Ngày đăng',
                                     'Ngày hết hạn',
                                     'Tên liên lạc',
                                     'Điện thoại',
                                     'Mobile',
                                     'Email',
                                     'Loại tin rao',
                                     'Địa chỉ',
                                     'Số phòng ngủ',
                                     'Số toilet',
                                     'Nội thất',
                                     'Hướng nhà',
                                     'Hướng ban công',
                                     'Content'
                                     ])
    return df

#============ NHA DAT CHO THUE ======================================================================
# def get_content_NhaDatChoThue(soup, attrsContent, attrsPropertyDetail, attrsProject, attrsContactDetail, attrsProjectDetail, attrsMoreInfo):
def get_content_NhaDatChoThue(soup):
    listData=[]
    d = {}
    
    # get more info
    attrsMoreInfo = 'prd-more-info'
    tableMoreInfo = soup.find('div', attrs = {'class':attrsMoreInfo}) 
    if not tableMoreInfo:
        print('tableMoreInfo is empty')
    else:
        moreInfo = get_more_info(tableMoreInfo)
        d.update(moreInfo)
    
    # get Contact Detail
    attrsContactDetail ='divCustomerInfo'
    tableContactDetail = soup.find('div', attrs = {'id':attrsContactDetail})
    if not tableContactDetail:
        print('tableContactDetail is empty')
    else:
        contactDetail = get_detail_info(tableContactDetail,'left','right')
        if contactDetail.get('Email')!= None:
            contactDetail['Email'] = tableContactDetail.find('div', id = 'contactEmail').a.text
            d.update(contactDetail)
        else:
            print('contactDetail is empty')
# print(d)    
    listData.append(d)
    return listData


In [2]:
#==================== RUN ==============================
#******* Nha dat cho thue *************************
url_base = 'https://batdongsan.com.vn'
u1_attrs1 = 'search-productItem'
u1_attrs2 = 'p-title'
listData = []
listUrl = []

for i in range(0,100):
    # loop through all the page
    url = str(url_base + '/nha-dat-cho-thue/p' + str(i))
    
    print('===================== Page '+ str(i) +' ===================================================================')
    print(url)
    
    #Step 1: get page content
    s = request_page(url)
    #print(s)
    # s = get_content_using_Selenium(u1)
    sleep(randint(8,16))

    #Step 2: get list of posts on each page
    listPost = get_list_of_posts(s, u1_attrs1, u1_attrs2)
    

    #Step 3: create a list of individual link to each post
    listIndividualLink = create_individual_link(url_base, listPost)

    # get list of individual link for each post in each page
    dic = {}
    dic['page'+ str(i)] = listIndividualLink
    listUrl.append(dic)

    #Step 4: get content
    for i in listIndividualLink:
        print(i)

        # loop through each page and get detail
        soup = get_content_using_Selenium(i) 

        # Pause the loop
        sleep(randint(3,3))

        #listData.append(get_content_NhaDatChoThue(soup, attrsContent, attrsPropertyDetail, attrsProject, attrsProjectDetail, attrsContactDetail, attrsMoreInfo))
        listData.append(get_content_NhaDatChoThue(soup))
              
              

===================== Page 0 ===================================================================
https://batdongsan.com.vn/nha-dat-cho-thue/p0
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-dien-bien-phu-phuong-22-prj-vinhomes-central-park/chuyen-ch-1-2-3-4pn-landmark-81-gia-tot-nhat-0977771919-pr25960584
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-3-2-2-phuong-12-5-prj-hado-centrosa-garden/1-2-3-4pn-ha-do-ntcb-den-full-noi-that-view-cuc-dep-bao-phi-quan-ly-pr25960096
https://batdongsan.com.vn/cho-thue-cua-hang-ki-ot-duong-a4-phuong-12-10/chinh-chu-mat-bang-2-mat-tien-150m2-thich-hop-kd-sieu-thi-cafe-quan-an-pr25958299
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-canh-phuong-22-prj-vinhomes-central-park/chuyen-vinmes-1-2-3-4-pn-pentuse-gia-tot-nhat-lh-0899303716-pr25959195
https://batdongsan.com.vn/cho-thue-cua-hang-ki-ot-xa-duong-xa-prj-vinhomes-ocean-park-gia-lam/chinh-chu-shop-de-vi-tri-dac-dia-nhat-tu-15-trieu-pr25957501
https://batdong

https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-trai-phuong-thuong-dinh-prj-royal-city/chinh-chu-r43122-day-du-noi-that-gia-18-5-trieu-lh-0989163656-pr25932040
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-15b-phuong-phu-my-9-prj-the-era-town/chuyen-dep-gia-re-tai-quan-7-2pn-full-nt-chi-8-5tr-0823023068-pr25934971
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-pho-ton-duc-thang-prj-vinhomes-golden-river-ba-son/cuc-t-vinme-central-park-1pn-4pn-bao-phi-internet-tro-chuyen-nha-lh-0987960930-pr25722915
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-trai-phuong-thuong-dinh-prj-royal-city/chuyen-ban-cat-lo-xem-nha-ngay-24-7-lien-he-0944-266-333-0946-053-050-pr9031384
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-pho-ton-duc-thang-phuong-ben-nghe-prj-vinhomes-golden-river-ba-son/1-pn-15tr-2pn-18-tr-3pn-28-tr-4pn-46-tr-penhoues-90-tr-villa-110-tr-pr25767013
contactDetail is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguye

contactDetail is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-vanh-dai-2-phuong-tam-binh-prj-sai-gon-avenue/saigon-2pn-6-5tr-thang-3pn-tu-7tr-thang-gia-tl-mua-dich-lh-0904-722-271-pr25728260
contactDetail is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-vanh-dai-2-phuong-tam-binh-prj-sai-gon-avenue/saigon-62m2-gia-7tr-co-noi-that-co-ban-cuc-dep-loc-khach-moi-o-0395442995-pr25284660
contactDetail is empty
https://batdongsan.com.vn/cho-thue-van-phong-duong-nguyen-thi-minh-khai-phuong-da-kao/moi-mat-tien-2-chieu-quan-1-san-dep-moi-do-oto-thoai-mai-he-thong-pccc-hien-dai-pr25891925
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-pho-ton-duc-thang-phuong-ben-nghe-prj-vinhomes-golden-river-ba-son/1pn-2pn-3pn-4pn-view-g-noi-that-chau-au-o-ngay-0977771919-pr25891201
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-tho-xa-phuoc-kieng-prj-saigon-south-residences/quan-ly-nhieu-residence-phu-my-hung-gia-tu-10tr-th-loan-091-898-1208-pr2589093

https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-canh-phuong-22-prj-vinhomes-central-park/ngay-4pn-ntcb-153m2-khu-gia-tot-lh-0967700703-pr25420827
https://batdongsan.com.vn/cho-thue-van-phong-duong-nguyen-canh-di-phuong-dai-kim-prj-khu-do-thi-dai-kim/mat-bang-mo-quan-cafe-hoac-day-du-noi-that-tai-nha-so-14-tt3-hm-hn-pr24486081
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-tho-prj-saigon-south-residences/ch-sai-gon-3pn-full-noi-that-cao-cap-moi-100-gia-16tr-th-lh-0911763318-pr25852191
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-pho-trung-liet-phuong-trung-liet/chinh-chu-khep-kin-tai-toa-nha-118-p-moi-an-thien-tien-nghi-day-du-pr14730517
https://batdongsan.com.vn/cho-thue-van-phong-duong-duy-tan-phuong-dich-vong-hau/gia-re-tai-tran-thai-tong-lien-he-0969526889-pr25684668
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-tran-duy-hung-phuong-trung-hoa-4-prj-vinhomes-dcapitale/chuyen-vinmes-cau-giay-re-nhat-lh-0968868588-pr25650354
htt

https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi-phuong-thao-dien-prj-masteri-thao-dien/hang-doc-quyen-chinh-chu-2pn-lau-cao-view-mat-gia-chi-13-5-trieu-pr25825514
https://batdongsan.com.vn/cho-thue-van-phong-duong-nguyen-van-troi-68/vip-tai-quan-phu-nhuan-lh-0986272828-pr24585948
contactDetail is empty
https://batdongsan.com.vn/cho-thue-van-phong-duong-nguyen-thi-thap-phuong-tan-quy/210-000vnd-m2-thang-toa-nha-green-country-q7-gan-lottte-sunrise-city-pr19327483
===================== Page 9 ===================================================================
https://batdongsan.com.vn/nha-dat-cho-thue/p9
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-tho-phuong-tan-hung-14-prj-sunrise-city-view/1-phong-ngu-7-5-trieu-du-noi-that-10-trieu-0909220855-pr25964670
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-au-co-phuong-9-13/28m2-gio-tu-do-tai-184-10-p9-q-tan-binh-gia-chi-tu-2-5tr-th-0914280675-pr25784809
https://batdongsan.com.vn/cho-th

...

https://batdongsan.com.vn/cho-thue-nha-rieng-duong-hiep-binh-phuong-hiep-binh-chanh/nguyen-can-moi-xay-hem-xe-hoi-pr25918727
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-phu-thuan-phuong-phu-thuan-3/giam-soc-mua-covid-trung-tam-q7-chi-tu-2-5tr-co-gac-lung-lh-0775512760-pr25943462
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-anh-dao-6-phuong-viet-hung-2-prj-vinhomes-riverside/tong-hop-cac-can-biet-thu-thang-6-2020-pr25814787
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-6c-xa-binh-hung-1-prj-khu-dan-cu-trung-son/co-n-that-may-lanh-cua-so-lon-va-thoang-o-kdc-t-gan-him-lam-dh-tdt-thuan-tien-qua-q1-pr25715498
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-ly-thai-to-phuong-1-14/mini-cao-cap-thoang-mat-gan-chung-cu-nguyen-thien-thuat-pr24872021
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-nguyen-bac-1-phuong-3-22/rong-ban-cong-san-tien-nghi-mat-me-gan-pham-van-hai-pr19606685
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-n

contactDetail is empty
===================== Page 92 ===================================================================
https://batdongsan.com.vn/nha-dat-cho-thue/p92
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-bang-a-phuong-hoang-liet-prj-tay-nam-ho-linh-dam/2-can-lien-ke-biet-thu-dt-100m2-va-260m2-gia-25tr-thang-da-hoan-thien-dep-pr25498985
contactDetail is empty
https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-dan-chu-phuong-binh-tho-prj-moonlight-residences/thu-duc-gia-tot-t-hien-nay-pr23322541
https://batdongsan.com.vn/cho-thue-cua-hang-ki-ot-duong-3-8-phuong-truong-tho-2-prj-lavita-garden/can-goc-shophouse-thu-duc-chi-22-trieu-thang-pr25347626
https://batdongsan.com.vn/cho-thue-nha-mat-pho-pho-lo-duc-phuong-dong-mac/dt-126m2-xay-7-tang-1-ham-cau-thang-may-pr25557295
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-go-o-moi-phuong-phu-thuan-3/xe-hoi-2-phong-ngu-lon-co-may-lanh-gan-gia-9-trieu-pr25938785
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-phan-va

https://batdongsan.com.vn/cho-thue-van-phong-duong-nguyen-van-cu-1-53/toa-nha-nc-mt-quan-1-ham-9-lau-8x30m-1300m2-gia-300tr-th-tl-0931343606-pr25937570
https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-thai-thi-nhan-phuong-10-10-prj-bau-cat-ii/sang-quan-cafe-banh-tai-khu-chung-cu-2-tan-binh-lh-0931232973-pr25921893
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-vo-chi-cong-phuong-phu-huu-2-prj-mega-ruby/shop-house-5x18m-tien-kinh-doanh-mua-ban-hoac-lam-vp-ty-mt-20m-canh-cc-safira-pr25365481
https://batdongsan.com.vn/cho-thue-van-phong-duong-le-duc-tho-phuong-my-dinh-2/toi-chinh-chu-san-pho-200m2-lh-truc-tiep-0977395962-pr25937536
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-van-troi-phuong-8-13-prj-kingston-residence/kingxston-dt-82m2-2pn-full-nt-gia-16tr-thag-view-dep-lh-0909503687-pr25937437
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-3-2-2-phuong-12-5-prj-hado-centrosa-garden/chinh-chu-chcc-ha-do-dt-60m2-1pn-khu-moi-view-dep-gia-14tr-lh-0936000

https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-bui-xuong-trach-phuong-dinh-cong-prj-chung-cu-c14-bo-quoc-phong/2pn-dt-65m2-png-362-ang-mai-pr25649512
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-chu-huy-man-phuong-phuc-dong-prj-hope-residence/70m2-gia-5tr-lh-032876990-pr25934037
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-hoa-binh-1-phuong-hiep-tan-prj-can-ho-richstar/gap-chcc-q-phu-65m2-2pn-2wc-gia-9-trieu-thang-lh-0931447274-gap-trang-pr25932043
===================== Page 97 ===================================================================
https://batdongsan.com.vn/nha-dat-cho-thue/p97
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-bui-xuong-trach-phuong-dinh-cong-prj-chung-cu-c14-bo-quoc-phong/2pn-dt-65m2-png-362-ang-mai-pr25649512
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-chu-huy-man-phuong-phuc-dong-prj-hope-residence/70m2-gia-5tr-lh-032876990-pr25934037
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-hoa-binh-

https://batdongsan.com.vn/cho-thue-van-phong-duong-nguyen-truong-to-phuong-12/a-nha-hoac-tung-tang-quan-4-0918219887-pr25927962
https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-nguyen-van-linh-phuong-tan-phong-9-prj-khu-nha-pho-hung-gia/nhieu-bang-re-phu-my-lien-he-chinh-chu-0901426119-pr25917787
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-mai-chi-tho-prj-sadora-apartment/gio-hang-sala-gia-tot-khach-dai-han-pr25928237
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-khuong-dinh-phuong-ha-dinh/mini-gan-nga-tu-so-lh-0905091428-pr18190147
contactDetail is empty
https://batdongsan.com.vn/cho-thue-van-phong-duong-xuan-dinh-phuong-xuan-tao-prj-khu-ngoai-giao-doan/mat-bang-tien-kinh-h-hoac-lam-gia-100-nghin-m2-pr25545935
contactDetail is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-mai-chi-tho-phuong-binh-khanh-prj-new-city-thu-thiem/3pn-noi-that-co-ban-17-trieu-th-bao-phi-quan-ly-lh-0909931237-pr24467005
https://batdongsan.com.vn/cho-thue-nha-rieng-

In [3]:
import pandas as pd
data100 = listData
a = create_dataFrame(data100)
a.to_csv("data0_100-2ndt++.csv", sep=',', encoding='utf-8')

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome(r"C:\Users\Nguyen Gia Bao\Desktop\chromedriver_win32\chromedriver")
driver.get("https://batdongsan.com.vn/nha-dat-ban")

driver.get("https://batdongsan.com.vn/ban-nha-rieng-duong-phung-khac-khoan-phuong-pham-dinh-ho/chinh-chu-can-mat-pho-khoan-pr25438308")
for title in driver.find_elements_by_id("divCustomerInfo"):
    print("==================================================")
    print("Title :"+title.text+"          " )
   

for title in driver.find_elements_by_class_name("prd-more-info"):
    print("Title :"+title.text+"          " )
    print("==================================================")
   


In [ ]:
!ls